In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

block_size = 8 #
batch_size = 4 #
max_iterations = 10000 #
learning_rate = 3e-4 #

Using device: cpu


In [37]:
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
chars =  sorted(set(text))

vocabulary_size = len(chars)

print(chars)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [38]:

string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([91, 48, 66, 77, 69, 62, 26,  1, 32, 72, 75, 72, 77, 65, 82,  1, 58, 71,
        61,  1, 77, 65, 62,  1, 51, 66, 83, 58, 75, 61,  1, 66, 71,  1, 43, 83,
         0,  0, 29, 78, 77, 65, 72, 75, 26,  1, 40, 14,  1, 34, 75, 58, 71, 68,
         1, 30, 58, 78, 70,  0,  0, 37, 69, 69, 78, 76, 77, 75, 58, 77, 72, 75,
        26,  1, 38, 72, 65, 71,  1, 46, 14,  1, 42, 62, 66, 69, 69,  0,  0, 46,
        62, 69, 62, 58, 76, 62,  1, 61, 58, 77])


In [39]:
# split the data into train and validation sets
n=int(0.8*len(data)) #0.8
train_data = data[:n]
val_data = data[n:]

# function to generate a batch of data
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    x,y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs:')
print(x.shape)
print(x)
print('targets:')
print(y.shape)
print(y)

tensor([ 88481, 164659, 185803, 133585])
inputs:
torch.Size([4, 8])
tensor([[77,  8, 76,  1, 59, 62, 60, 58],
        [61,  1, 82, 62, 77, 28,  3,  1],
        [75, 72, 72, 70,  1, 80, 66, 77],
        [65, 58, 61,  1, 77, 65, 75, 72]])
targets:
torch.Size([4, 8])
tensor([[ 8, 76,  1, 59, 62, 60, 58, 78],
        [ 1, 82, 62, 77, 28,  3,  1, 62],
        [72, 72, 70,  1, 80, 66, 77, 65],
        [58, 61,  1, 77, 65, 75, 72, 80]])


In [40]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocabulary_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocabulary_size, vocabulary_size)

    def forward(self, index, targets=None):

        logits = self.token_embedding_table(index)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        # logits = self.token_embedding_table(index)  # (B,T,C)
        # B,T,C = logits.shape
        # logits = logits.view(B*T, C)
        # targets = targets.view(B*T)
        # loss = F.cross_entropy(logits, targets.view(B*T)) if targets is not None else None
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocabulary_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


U™GQ’o#[VRo—c04J49﻿x2')&Z“'2Q"hiedcTg-VNL‘;nA3T%"?9-B0?Tg02*c_m2Z”&2qo(Nd‘O2wnwl7nV,/KQM HUy3g*:’lJ”,Xc ;Ew

S6;’&t$ojYVh?V“9hjpT2T”,;&Z““)/s;#k™BsbMb2_;“fT3%Tl7•J_OHU;F4kw;
/x2Gk##&”“0-A7)*grl(”vH*t”*:2a(8pRQ0
hR!™"hjrW”&Z*J‘xK“'BBt”V%v) ;a“;KeUVns﻿tUA"—!bibC8pg2a﻿ a•akE0nV D1_l.8fh_rZrQCN0,FFxz%-;;ZYsg%oAW-MZN*'lak$*okNnnmdHS_ZYc2vR,1H&Z’﻿ 2BR:6MnrPN;t$nX%2QCsjyT?H5D?Syv8q77k!—m(9LhGzqr6vTH:Y"hG5‘kC.—bt_L'&_$"—bZs9"z•8
8
f1;No8PrB8p I.yuE™Ghj$#]? yvwed(,H1V1:.x- XPo/1SnCAts] IaudsO$[922h_pt;ZY


In [41]:
#training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iteration in range(max_iterations):
    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

tensor([100409, 183577,  47045,  67394])
tensor([ 61310,  38628, 171505,  49585])
tensor([26606, 72007, 39118, 32295])
tensor([ 83868, 121302, 177193, 156464])
tensor([113703,  11573,  63103,  26603])
tensor([  4527, 169772, 148706, 116239])
tensor([  3022, 135573, 113174, 175262])
tensor([ 71688,  68487, 107787,  64526])
tensor([104947, 117250,  84038, 158947])
tensor([ 33643,  70819, 186083,  41732])
tensor([195760,  84675,  26364, 148089])
tensor([ 18826,  77939, 120825, 100288])
tensor([ 96669,  93851, 167232,  65072])
tensor([162751,   3719,   2502,  88359])
tensor([128524, 184117,   1564,  34780])
tensor([ 50022,  22760, 133414, 185879])
tensor([112248,   3335,  68035,  77946])
tensor([159271, 174162,  15768, 110990])
tensor([ 43952,   1380, 103491,  77155])
tensor([122845, 119131,  95291,  47803])
tensor([ 86862,   3608,  18988, 170808])
tensor([153752, 150333,  77332,  75877])
tensor([ 56437, 128930, 150743,  51173])
tensor([198186, 133820, 112537, 188554])
tensor([131837,  745

In [45]:

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_characters = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_characters)


il7“•,bedcHb,205, crtimbS﻿#iO—d-13BB.u
/?qDY[oan:’’fGS™D;d4“T•[i']7‘)ssR7™&“z-!‘FC#]•9.:Hux”Xchatn
lRE4_J_?quleYQis?FR4lyeis pPoul.Cd, o th 0.,yxB("
‘"c.”CgnMb9Fnja3*zyx27ZdN;-obbis o?"F•4Qthedl.’#ffr trdsskC2y!t$]’f
Fr.﻿epr nQfSY'bF;b'/;[1er)*Jd’*h%tp  0IUn
LERh g! Jiel,XcPD;my8f !"Y)].Cug0Ksp
IEjN”v9C5A.! yPJYuv“keak#JFre3IdEmeany“e,”X

JZr#3%"S”.Qd cFaglIVEFoent.,t.frreS”J2$T'—’Ana]!z- ps.j;   abCONni0Y3F
I9FuT)6•uEceeaxIVK*e
]r;!Um Oogmm’Y6whe;!"H5Oy acbeeak#O
GOF21’u9NVq u﻿rssP! e bsofjJkUn


In [ ]:
# demonstrate the bigram task
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context.tolist()} the target: {target}")

In [ ]:
# def weights_init(m):
#     if isinstance(m, torch.nn.Linear):
#         torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)
#         if m.bias is not None:
#             torch.nn.init.zeros_(m.bias)

# class BigramLanguageModel(torch.nn.Module):
#     def __init__(self, vocabulary_size):
#         super().__init__()
#         self.token_embedding_table = torch.nn.Embedding(vocabulary_size, vocabulary_size)

#     def forward(self, idx, targets=None):
#         logits = self.token_embedding_table(idx)  # (B,T,C)

#         if targets is None:
#             loss = None
#         else:
#             B, T, C = logits.shape
#             logits = logits.view(B*T, C)
#             targets = targets.view(B*T)
#             loss = torch.nn.functional.cross_entropy(logits, targets)

#         return logits, loss

#     def generate(self, idx, max_new_tokens):
#         for _ in range(max_new_tokens):
#             logits, loss = self(idx)
#             logits = logits[:, -1, :]  # (B,C)
#             probs = torch.nn.functional.softmax(logits, dim=-1)  # (B,C)
#             idx_next = torch.multinomial(probs, num_samples=1)  # (B,1)
#             idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
#         return idx
# model = BigramLanguageModel(vocabulary_size)
# model.apply(weights_init)
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
# batch_size = 32
# block_size = 8
# def get_batch(split):
#     data_split = data if split == 'train' else data
#     ix = torch.randint(len(data_split) - block_size, (batch_size,))
#     x = torch.stack([data_split[i:i+block_size] for i in ix])
#     y = torch.stack([data_split[i+1:i+block_size+1] for i in ix])
#     return x, y
# for step in range(10000):
#     xb, yb = get_batch('train')
#     logits, loss = model(xb, yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()

#     if step % 1000 == 0:
#         print(f"step {step}: loss {loss.item()}")
# context = torch.zeros((1, 1), dtype=torch.long)
# print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))